In [7]:
%matplotlib inline
import torch
from sebm.cebm_sgld import SGLD_sampler, init_cebm
dataset =  'cifar10' # 'svhn' # 'cifar10' # 'mnist' #  'flowers102' #
if dataset == 'mnist' or dataset =='fashionmnist':
    input_channels, im_height, im_width = 1, 28, 28
else:
    input_channels, im_height, im_width = 3, 32, 32
device = torch.device('cuda:1')
arch =  'simplenet2' # 'wresnet' # 'simplenet'
depth, width = 28, 10
heldout = -1
ss = 2
seed = 1
lr = 5e-5
optimize_priors = False
channels, kernels, strides, paddings =[64,64,32,32], [3,4,4,4], [1,2,2,2], [1,1,1,1]
hidden_dim, latent_dim = [128], 128
data_noise_std = 3e-2
sgld_noise_std, sgld_lr, sgld_num_steps = 7.5e-3, 2.0, 60
reg_alpha =  5e-3
activation = 'Swish'
buffer_init, buffer_dup_allowed = True, True
data_dir = '../../../sebm_data/'
load_version = 'cebm_%sss-out=%s-d=%s-seed=%d-lr=%s-zd=%d-d_ns=%s-sgld-ns=%s-lr=%s-steps=%s-reg=%s-act=%s-arch=%s' % (ss, heldout, dataset, seed, lr, latent_dim, data_noise_std, sgld_noise_std, sgld_lr, sgld_num_steps,  reg_alpha, activation, arch)
if arch == 'wresnet':
    ebm = init_cebm(arch=arch,
                    ss=ss,
                    optimize_priors=optimize_priors,
                    device=device,
                    depth=depth,
                    width=width,
                    hidden_dim=hidden_dim,
                    latent_dim=latent_dim,
                    act=activation)
elif arch == 'simplenet' or arch == 'simplenet2':
    ebm = init_cebm(arch=arch,
                    ss=ss,
                    optimize_priors=optimize_priors,
                    device=device,
                    im_height=im_height, 
                    im_width=im_width, 
                    input_channels=input_channels, 
                    channels=channels, 
                    kernels=kernels, 
                    strides=strides, 
                    paddings=paddings, 
                    hidden_dim=hidden_dim,
                    latent_dim=latent_dim,
                    activation=activation)
ebm = ebm.cuda().to(device)
print('Loading trained weights..')
ebm.load_state_dict(torch.load('../weights/cp-%s' % load_version)['model_state_dict'])

Initialize Model=CEBM_2ss...
Loading trained weights..


<All keys matched successfully>

In [8]:
from sebm.eval import Evaluator_EBM
evaluator = Evaluator_EBM(ebm, device, dataset, data_dir, data_noise_std=1e-2)

In [9]:
# evaluator.plot_post_under_prior(train=False, density=False) #save_name='cebm_%s' % dataset)

In [10]:
# evaluator.train_logistic_classifier()

In [11]:
images_ebm, list_samples = evaluator.uncond_sampling(batch_size=100, 
                                          sgld_steps=1000,
                                          sgld_lr=4.0,
                                          sgld_noise_std=1e-2,
                                          grad_clipping=False)
# evaluator.plot_samples(images_ebm, fs=6, save_name='samples')

In [12]:
for i in range(list_samples.shape[0]):
    evaluator.plot_samples(list_samples[i], fs=6, save_name='cebm_%s_%02d' % (dataset, i+1))